In [201]:
%matplotlib inline

Generation of Questions with a seq2seq network 
*************************************************************




::

    [KEY: > input, = target, < output]

   

This is made possible by the simple but powerful idea of the `sequence
to sequence network <http://arxiv.org/abs/1409.3215>`__, in which two
recurrent neural networks work together to transform one sequence to
another. An encoder network condenses an input sequence into a vector,
and a decoder network unfolds that vector into a new sequence.

.. figure:: /_static/img/seq-seq-images/seq2seq.png
   :alt:

To improve upon this model we'll use an `attention
mechanism <https://arxiv.org/abs/1409.0473>`__, which lets the decoder
learn to focus over a specific range of the input sequence.

**Requirements**



In [202]:
from __future__ import unicode_literals, print_function, division
from io import open
import string
import re
import random

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

Loading data files
===================

The data for this project is a set of many thousands of sentence pairs, broken down into two main groups i.e agreement and no agreement and are present in the dataset folder of the project



Similar to the character encoding used in the character-level RNN
tutorials, we will be representing each word in a language as a one-hot
vector, or giant vector of zeros except for a single one (at the index
of the word). Compared to the dozens of characters that might exist in a
language, there are many many more words, so the encoding vector is much
larger. We will however cheat a bit and trim the data to only use a few
thousand words per language.

.. figure:: /_static/img/seq-seq-images/word-encoding.png
   :alt:





We'll need a unique index per word to use as the inputs and targets of
the networks later. To keep track of all this we will use a helper class
called ``Lang`` which has word → index (``word2index``) and index → word
(``index2word``) dictionaries, as well as a count of each word
``word2count`` to use to later replace rare words.




In [203]:
SOS_token = 0
IDENT_TOKEN = -1
QUEST_TOKEN = -1
EOS_TOKEN =1


class Lang:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "SOS", 1: "EOS"}
        self.n_words = 2  # Count SOS and EOS

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

we will make evertything lower case adn trim most punctuation.




In [204]:


# Lowercase, trim, and remove non-letter characters


def normalizeString(s):
    s = (s.lower().strip())
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)
    return s

To read the data file we will split the file into lines, and then split
lines into pairs




In [205]:
def readLangs(lang1, lang2, reverse=False):
    print("Reading lines...")

    # Read the file and split into lines
    lines = open('./../data/agreement_data/train_data.txt' ,).\
        read().strip().split('\n')

    # Split every line into pairs and normalize
    pairs = [[normalizeString(s) for s in l.split('\t')] for l in lines]

    # Reverse pairs, make Lang instances
    if reverse:
        pairs = [list(reversed(p)) for p in pairs]
        input_lang = Lang(lang2)
        output_lang = Lang(lang1)
    else:
        input_lang = Lang(lang1)
        output_lang = Lang(lang2)

    return input_lang, output_lang, pairs

The maximum length of each sentence has been set to a 100 words.



In [206]:
MAX_LENGTH = 100



def filterPairs(pairs):
    return [pair for pair in pairs]

The full process for preparing the data is:

-  Read text file and split into lines, split lines into pairs
-  Normalize text, filter by length and content
-  Make word lists from sentences in pairs




In [207]:
def prepareData(lang1, lang2, reverse=False):
    input_lang, output_lang, pairs = readLangs(lang1, lang2, reverse)
    print("Read %s sentence pairs" % len(pairs))
    pairs = filterPairs(pairs)
    print("Trimmed to %s sentence pairs" % len(pairs))
    print("Counting words...")
    for pair in pairs:
        input_lang.addSentence(pair[0])
        output_lang.addSentence(pair[1])
    print("Counted words:")
    print(input_lang.name, input_lang.n_words)
    print(output_lang.name, output_lang.n_words)
    return input_lang, output_lang, pairs


input_lang, output_lang, pairs = prepareData('out', 'inp', True)
IDENT_TOKEN = input_lang.word2index["ident"]
QUEST_TOKEN = input_lang.word2index["quest"]
print(random.choice(pairs))

Reading lines...
Read 435500 sentence pairs
Trimmed to 435500 sentence pairs
Counting words...
Counted words:
inp 54
out 52
['her rabbit by your elephants does sleep ident', 'her rabbit by your elephants does sleep']


The Seq2Seq Model
=================

A Recurrent Neural Network, or RNN, is a network that operates on a
sequence and uses its own output as input for subsequent steps.

A `Sequence to Sequence network <http://arxiv.org/abs/1409.3215>`__, or
seq2seq network, or `Encoder Decoder
network <https://arxiv.org/pdf/1406.1078v3.pdf>`__, is a model
consisting of two RNNs called the encoder and decoder. The encoder reads
an input sequence and outputs a single vector, and the decoder reads
that vector to produce an output sequence.

.. figure:: /_static/img/seq-seq-images/seq2seq.png
   :alt:

Unlike sequence prediction with a single RNN, where every input
corresponds to an output, the seq2seq model frees us from sequence
length and order, which makes it ideal for translation between two
languages.

With a seq2seq model the encoder creates a single vector which, in the
ideal case, encodes the "meaning" of the input sequence into a single
vector — a single point in some N dimensional space of sentences.




The Encoder
-----------

The encoder of a seq2seq network is a RNN that outputs some value for
every word from the input sentence. For every input word the encoder
outputs a vector and a hidden state, and uses the hidden state for the
next input word.

.. figure:: /_static/img/seq-seq-images/encoder-network.png
   :alt:





In [208]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output = embedded
        output, hidden = self.gru(output, hidden)
        return output, hidden

    def initHidden(self):
        return torch.randn(1, 1, self.hidden_size, device=device)

The Decoder
-----------

The decoder is another RNN that takes the encoder output vector(s) and
outputs a sequence of words to create the translation.




Simple Decoder
^^^^^^^^^^^^^^

In the simplest seq2seq decoder we use only last output of the encoder.
This last output is sometimes called the *context vector* as it encodes
context from the entire sequence. This context vector is used as the
initial hidden state of the decoder.

At every step of decoding, the decoder is given an input token and
hidden state. The initial input token is the start-of-string ``<SOS>``
token, and the first hidden state is the context vector (the encoder's
last hidden state).

.. figure:: /_static/img/seq-seq-images/decoder-network.png
   :alt:





In [209]:
class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(output_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        output = self.embedding(input).view(1, 1, -1)
        output = F.relu(output)
        output, hidden = self.gru(output, hidden)
        output = self.softmax(self.out(output[0]))
        return output, hidden

    def initHidden(self):
        return torch.randn(1, 1, self.hidden_size, device=device)

Attention Decoder
^^^^^^^^^^^^^^^^^

If only the context vector is passed betweeen the encoder and decoder,
that single vector carries the burden of encoding the entire sentence.

Attention allows the decoder network to "focus" on a different part of
the encoder's outputs for every step of the decoder's own outputs. First
we calculate a set of *attention weights*. These will be multiplied by
the encoder output vectors to create a weighted combination. The result
(called ``attn_applied`` in the code) should contain information about
that specific part of the input sequence, and thus help the decoder
choose the right output words.

.. figure:: https://i.imgur.com/1152PYf.png
   :alt:

Calculating the attention weights is done with another feed-forward
layer ``attn``, using the decoder's input and hidden state as inputs.
Because there are sentences of all sizes in the training data, to
actually create and train this layer we have to choose a maximum
sentence length (input length, for encoder outputs) that it can apply
to. Sentences of the maximum length will use all the attention weights,
while shorter sentences will only use the first few.

.. figure:: /_static/img/seq-seq-images/attention-decoder-network.png
   :alt:





In [210]:
class AttnDecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, dropout_p=0.1, max_length=MAX_LENGTH):
        super(AttnDecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.dropout_p = dropout_p
        self.max_length = max_length

        self.embedding = nn.Embedding(self.output_size, self.hidden_size)
        self.attn = nn.Linear(self.hidden_size * 2, self.max_length)
        self.attn_combine = nn.Linear(self.hidden_size * 2, self.hidden_size)
        self.dropout = nn.Dropout(self.dropout_p)
        self.gru = nn.GRU(self.hidden_size, self.hidden_size)
        self.out = nn.Linear(self.hidden_size, self.output_size)

    def forward(self, input, hidden, encoder_outputs):
        embedded = self.embedding(input).view(1, 1, -1)
        embedded = self.dropout(embedded)

        attn_weights = F.softmax(
            self.attn(torch.cat((embedded[0], hidden[0]), 1)), dim=1)
        attn_applied = torch.bmm(attn_weights.unsqueeze(0),
                                 encoder_outputs.unsqueeze(0))

        output = torch.cat((embedded[0], attn_applied[0]), 1)
        output = self.attn_combine(output).unsqueeze(0)

        output = F.relu(output)
        output, hidden = self.gru(output, hidden)

        output = F.log_softmax(self.out(output[0]), dim=1)
        return output, hidden, attn_weights

    def initHidden(self):
        return torch.randn(1, 1, self.hidden_size, device=device)



Training
========

Preparing Training Data
-----------------------

To train, for each pair we will need an input tensor (indexes of the
words in the input sentence) and target tensor (indexes of the words in
the target sentence). While creating these vectors we will append the
EOS token to both sequences.




In [211]:
def indexesFromSentence(lang, sentence):
    return [lang.word2index[word] for word in sentence.split(' ')]


def tensorFromSentence(lang, sentence):
    indexes = indexesFromSentence(lang, sentence)
    indexes.append(EOS_TOKEN)
        
    return torch.tensor(indexes, dtype=torch.long, device=device).view(-1, 1)


def tensorsFromPair(pair):
    input_tensor = tensorFromSentence(input_lang, pair[0])
    target_tensor = tensorFromSentence(output_lang, pair[1])
    return (input_tensor, target_tensor)

Training the Model
------------------

To train we run the input sentence through the encoder, and keep track
of every output and the latest hidden state. Then the decoder is given
the ``<SOS>`` token as its first input, and the last hidden state of the
encoder as its first hidden state.

"Teacher forcing" is the concept of using the real target outputs as
each next input, instead of using the decoder's guess as the next input.
Using teacher forcing causes it to converge faster but `when the trained
network is exploited, it may exhibit
instability <http://minds.jacobs-university.de/sites/default/files/uploads/papers/ESNTutorialRev.pdf>`__.

You can observe outputs of teacher-forced networks that read with
coherent grammar but wander far from the correct translation -
intuitively it has learned to represent the output grammar and can "pick
up" the meaning once the teacher tells it the first few words, but it
has not properly learned how to create the sentence from the translation
in the first place.

Because of the freedom PyTorch's autograd gives us, we can randomly
choose to use teacher forcing or not with a simple if statement. Turn
``teacher_forcing_ratio`` up to use more of it.




In [212]:
tensorsFromPair(pairs[0])

(tensor([[2],
         [3],
         [4],
         [5],
         [6],
         [1]]), tensor([[2],
         [3],
         [4],
         [5],
         [1]]))

In [213]:
teacher_forcing_ratio = 0.5


def train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, max_length=MAX_LENGTH):
    encoder_hidden = encoder.initHidden()

    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)

    encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

    loss = 0

    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(
            input_tensor[ei], encoder_hidden)
        encoder_outputs[ei] = encoder_output[0, 0]

    decoder_input = torch.tensor([[SOS_token]], device=device)

    decoder_hidden = encoder_hidden

    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    if use_teacher_forcing:
        # Teacher forcing: Feed the target as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            loss += criterion(decoder_output, target_tensor[di])
            decoder_input = target_tensor[di]  # Teacher forcing

    else:
        # Without teacher forcing: use its own predictions as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            topv, topi = decoder_output.topk(1)
            decoder_input = topi.squeeze().detach()  # detach from history as input

            loss += criterion(decoder_output, target_tensor[di])
            if decoder_input.item() == IDENT_TOKEN or decoder_input.item() == QUEST_TOKEN or decoder_input.item()== EOS_TOKEN:
                break

    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / target_length

This is a helper function to print time elapsed and estimated time
remaining given the current time and progress %.




In [214]:
import time
import math


def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

The whole training process looks like this:

-  Start a timer
-  Initialize optimizers and criterion
-  Create set of training pairs
-  Start empty losses array for plotting

Then we call ``train`` many times and occasionally print the progress (%
of examples, time so far, estimated time) and average loss.




In [191]:
def trainIters(encoder, decoder, n_iters, print_every=1000, plot_every=100, learning_rate=0.01):
    start = time.time()
    plot_losses = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every

    encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)
    training_pairs = [tensorsFromPair(random.choice(pairs))
                      for i in range(n_iters)]
    criterion = nn.NLLLoss()

    for iter in range(1, n_iters + 1):
        training_pair = training_pairs[iter - 1]
        input_tensor = training_pair[0]
        target_tensor = training_pair[1]

        loss = train(input_tensor, target_tensor, encoder,
                     decoder, encoder_optimizer, decoder_optimizer, criterion)
        print_loss_total += loss
        plot_loss_total += loss

        if iter % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('%s (%d %d%%) %.4f' % (timeSince(start, iter / n_iters),
                                         iter, iter / n_iters * 100, print_loss_avg))

        if iter % plot_every == 0:
            plot_loss_avg = plot_loss_total / plot_every
            plot_losses.append(plot_loss_avg)
            plot_loss_total = 0

    showPlot(plot_losses)

Plotting results
----------------

Plotting is done with matplotlib, using the array of loss values
``plot_losses`` saved while training.




In [192]:
import matplotlib.pyplot as plt
plt.switch_backend('agg')
import matplotlib.ticker as ticker
import numpy as np


def showPlot(points):
    plt.figure()
    fig, ax = plt.subplots()
    # this locator puts ticks at regular intervals
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)

Evaluation
==========

Evaluation is mostly the same as training, but there are no targets so
we simply feed the decoder's predictions back to itself for each step.
Every time it predicts a word we add it to the output string, and if it
predicts the EOS token we stop there. We also store the decoder's
attention outputs for display later.




In [193]:
def evaluate(encoder, decoder, sentence, max_length=MAX_LENGTH):
    with torch.no_grad():
        input_tensor = tensorFromSentence(input_lang, sentence)
        input_length = input_tensor.size()[0]
        encoder_hidden = encoder.initHidden()

        encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

        for ei in range(input_length):
            encoder_output, encoder_hidden = encoder(input_tensor[ei],
                                                     encoder_hidden)
            encoder_outputs[ei] += encoder_output[0, 0]

        decoder_input = torch.tensor([[SOS_token]], device=device)  # SOS

        decoder_hidden = encoder_hidden

        decoded_words = []
        decoder_attentions = torch.zeros(max_length, max_length)

        for di in range(max_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            decoder_attentions[di] = decoder_attention.data
            topv, topi = decoder_output.data.topk(1)
            if topi.item() == IDENT_TOKEN :
                decoded_words.append('<IDENT>')
                break
            elif topi.item() == QUEST_TOKEN :
                decoded_words.append('<QUEST>')
                break
            elif topi.item() == EOS_TOKEN :
                decoded_words.append('<EOS>')
                break
            else:
                decoded_words.append(output_lang.index2word[topi.item()])

            decoder_input = topi.squeeze().detach()

        return decoded_words, decoder_attentions[:di + 1]

We can evaluate random sentences from the training set and print out the
input, target, and output to make some subjective quality judgements:




In [194]:
def evaluateRandomly(encoder, decoder, n=10):
    for i in range(n):
        pair = random.choice(pairs)
        print('>', pair[0])
        print('=', pair[1])
        output_words, attentions = evaluate(encoder, decoder, pair[0])
        output_sentence = ' '.join(output_words)
        print('<', output_sentence)
        print('')

Training and Evaluating
=======================



In [195]:
class TheModelClass(nn.Module):
    def __init__(self):
        super(TheModelClass, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# Initialize model
model = TheModelClass()

# Initialize optimizer
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# Print model's state_dict
print("Model's state_dict:")
for param_tensor in model.state_dict():
    print(param_tensor, "\t", model.state_dict()[param_tensor].size())

# Print optimizer's state_dict
print("Optimizer's state_dict:")
for var_name in optimizer.state_dict():
    print(var_name, "\t", optimizer.state_dict()[var_name])


Model's state_dict:
conv1.weight 	 torch.Size([6, 3, 5, 5])
conv1.bias 	 torch.Size([6])
conv2.weight 	 torch.Size([16, 6, 5, 5])
conv2.bias 	 torch.Size([16])
fc1.weight 	 torch.Size([120, 400])
fc1.bias 	 torch.Size([120])
fc2.weight 	 torch.Size([84, 120])
fc2.bias 	 torch.Size([84])
fc3.weight 	 torch.Size([10, 84])
fc3.bias 	 torch.Size([10])
Optimizer's state_dict:
state 	 {}
param_groups 	 [{'momentum': 0.9, 'params': [140310933905768, 140310933909152, 140310793436904, 140310793433160, 140310793435752, 140310793436256, 140310839989160, 140310793436400, 140310793437120, 140310793437048], 'lr': 0.001, 'nesterov': False, 'weight_decay': 0, 'dampening': 0}]


In [196]:
model_exists = False
if model_exists :
    model = torch.load("no_agreement.pt")
    model.eval()

In [197]:
hidden_size = 256
counter=1
while(counter == 1 ):
    name = "aagreement_"+str(counter)+".pt"
    print(name)
    
    encoder1 = EncoderRNN(input_lang.n_words, hidden_size).to(device)
    attn_decoder1 = AttnDecoderRNN(hidden_size, output_lang.n_words, dropout_p=0.1).to(device)
    trainIters(encoder1, attn_decoder1, 75000, print_every=50)
    
    torch.save(model.state_dict,name)   
    counter+=1

aagreement_1.pt
0m 2s (- 61m 49s) (50 0%) 2.9914
0m 3s (- 42m 31s) (100 0%) 2.5123
0m 4s (- 36m 42s) (150 0%) 2.7625
0m 5s (- 34m 7s) (200 0%) 2.9020
0m 6s (- 32m 48s) (250 0%) 2.7935
0m 7s (- 32m 9s) (300 0%) 2.6161
0m 8s (- 31m 25s) (350 0%) 2.4848
0m 9s (- 30m 39s) (400 0%) 2.3509
0m 10s (- 30m 0s) (450 0%) 2.4903
0m 12s (- 29m 57s) (500 0%) 2.4747
0m 13s (- 29m 27s) (550 0%) 1.9994
0m 14s (- 29m 14s) (600 0%) 2.4514
0m 15s (- 29m 1s) (650 0%) 2.5777
0m 16s (- 28m 34s) (700 0%) 2.2137
0m 17s (- 28m 22s) (750 1%) 2.3310
0m 18s (- 28m 14s) (800 1%) 2.1025
0m 19s (- 28m 1s) (850 1%) 2.1806
0m 20s (- 27m 48s) (900 1%) 2.1513
0m 21s (- 27m 43s) (950 1%) 2.2729
0m 22s (- 27m 43s) (1000 1%) 2.4233
0m 23s (- 27m 38s) (1050 1%) 2.2265
0m 24s (- 27m 29s) (1100 1%) 2.4179
0m 25s (- 27m 24s) (1150 1%) 2.2518
0m 26s (- 27m 16s) (1200 1%) 1.9391
0m 27s (- 27m 6s) (1250 1%) 2.0662
0m 28s (- 26m 58s) (1300 1%) 2.1858
0m 29s (- 26m 52s) (1350 1%) 2.3340
0m 30s (- 26m 41s) (1400 1%) 1.9544
0m 31s (- 

4m 27s (- 24m 54s) (11400 15%) 0.2721
4m 29s (- 24m 53s) (11450 15%) 0.2830
4m 30s (- 24m 53s) (11500 15%) 0.2737
4m 31s (- 24m 52s) (11550 15%) 0.2694
4m 32s (- 24m 52s) (11600 15%) 0.3132
4m 34s (- 24m 51s) (11650 15%) 0.2988
4m 35s (- 24m 51s) (11700 15%) 0.3669
4m 36s (- 24m 50s) (11750 15%) 0.3025
4m 38s (- 24m 49s) (11800 15%) 0.2495
4m 39s (- 24m 48s) (11850 15%) 0.2319
4m 40s (- 24m 47s) (11900 15%) 0.2504
4m 41s (- 24m 47s) (11950 15%) 0.3067
4m 43s (- 24m 46s) (12000 16%) 0.2810
4m 44s (- 24m 45s) (12050 16%) 0.2630
4m 45s (- 24m 45s) (12100 16%) 0.2997
4m 46s (- 24m 44s) (12150 16%) 0.2603
4m 48s (- 24m 43s) (12200 16%) 0.1975
4m 49s (- 24m 42s) (12250 16%) 0.1793
4m 50s (- 24m 41s) (12300 16%) 0.2568
4m 51s (- 24m 39s) (12350 16%) 0.3640
4m 52s (- 24m 38s) (12400 16%) 0.2397
4m 54s (- 24m 37s) (12450 16%) 0.2760
4m 55s (- 24m 37s) (12500 16%) 0.2510
4m 56s (- 24m 36s) (12550 16%) 0.3128
4m 57s (- 24m 35s) (12600 16%) 0.3185
4m 59s (- 24m 34s) (12650 16%) 0.2232
5m 0s (- 24m

9m 12s (- 21m 44s) (22300 29%) 0.2010
9m 13s (- 21m 43s) (22350 29%) 0.1044
9m 14s (- 21m 42s) (22400 29%) 0.1761
9m 16s (- 21m 42s) (22450 29%) 0.1492
9m 17s (- 21m 41s) (22500 30%) 0.0724
9m 18s (- 21m 40s) (22550 30%) 0.0916
9m 20s (- 21m 38s) (22600 30%) 0.1154
9m 21s (- 21m 37s) (22650 30%) 0.1377
9m 22s (- 21m 37s) (22700 30%) 0.0883
9m 24s (- 21m 36s) (22750 30%) 0.1011
9m 25s (- 21m 35s) (22800 30%) 0.1195
9m 26s (- 21m 33s) (22850 30%) 0.1464
9m 28s (- 21m 33s) (22900 30%) 0.0851
9m 29s (- 21m 31s) (22950 30%) 0.1140
9m 31s (- 21m 30s) (23000 30%) 0.1130
9m 32s (- 21m 29s) (23050 30%) 0.0839
9m 33s (- 21m 28s) (23100 30%) 0.1030
9m 34s (- 21m 27s) (23150 30%) 0.0903
9m 36s (- 21m 26s) (23200 30%) 0.1006
9m 37s (- 21m 25s) (23250 31%) 0.0984
9m 38s (- 21m 24s) (23300 31%) 0.0812
9m 40s (- 21m 23s) (23350 31%) 0.0700
9m 41s (- 21m 22s) (23400 31%) 0.0992
9m 42s (- 21m 20s) (23450 31%) 0.0814
9m 44s (- 21m 19s) (23500 31%) 0.0801
9m 45s (- 21m 18s) (23550 31%) 0.0833
9m 46s (- 21

13m 54s (- 17m 44s) (32950 43%) 0.0593
13m 55s (- 17m 43s) (33000 44%) 0.0531
13m 56s (- 17m 42s) (33050 44%) 0.0389
13m 58s (- 17m 41s) (33100 44%) 0.0423
13m 59s (- 17m 39s) (33150 44%) 0.0466
14m 0s (- 17m 38s) (33200 44%) 0.0438
14m 2s (- 17m 37s) (33250 44%) 0.0388
14m 3s (- 17m 36s) (33300 44%) 0.0350
14m 4s (- 17m 35s) (33350 44%) 0.0681
14m 6s (- 17m 33s) (33400 44%) 0.0407
14m 7s (- 17m 32s) (33450 44%) 0.0419
14m 8s (- 17m 31s) (33500 44%) 0.0696
14m 10s (- 17m 30s) (33550 44%) 0.0533
14m 11s (- 17m 29s) (33600 44%) 0.0534
14m 12s (- 17m 27s) (33650 44%) 0.0586
14m 14s (- 17m 26s) (33700 44%) 0.0509
14m 15s (- 17m 25s) (33750 45%) 0.0621
14m 16s (- 17m 24s) (33800 45%) 0.0643
14m 18s (- 17m 23s) (33850 45%) 0.0460
14m 19s (- 17m 21s) (33900 45%) 0.0549
14m 20s (- 17m 20s) (33950 45%) 0.1108
14m 22s (- 17m 19s) (34000 45%) 0.0462
14m 23s (- 17m 18s) (34050 45%) 0.0568
14m 24s (- 17m 17s) (34100 45%) 0.0288
14m 26s (- 17m 15s) (34150 45%) 0.0376
14m 27s (- 17m 14s) (34200 45%) 

KeyboardInterrupt: 

In [200]:
evaluateRandomly(encoder1, attn_decoder1)

> the unicorn who the bird does sleep does sleep quest
= does the unicorn who the bird does sleep sleep


NameError: name 'EOS_TOKEM' is not defined

Visualizing Attention
---------------------

A useful property of the attention mechanism is its highly interpretable
outputs. Because it is used to weight specific encoder outputs of the
input sequence, we can imagine looking where the network is focused most
at each time step.

You could simply run ``plt.matshow(attentions)`` to see attention output
displayed as a matrix, with the columns being input steps and rows being
output steps:




In [45]:
output_words, attentions = evaluate(
    encoder1, attn_decoder1, "our cats do read quest")
plt.matshow(attentions.numpy())

For a better viewing experience we will do the extra work of adding axes
and labels:




In [73]:
def showAttention(input_sentence, output_words, attentions):
    # Set up figure with colorbar
    fig = plt.figure()
    ax = fig.add_subplot(111)
    cax = ax.matshow(attentions.numpy(), cmap='bone')
    fig.colorbar(cax)

    # Set up axes
    ax.set_xticklabels([''] + input_sentence.split(' ') +
                       ['<end>'], rotation=90)
    ax.set_yticklabels([''] + output_words)

    # Show label at every tick
    ax.xaxis.set_major_locator(ticker.MultipleLocator(1))
    ax.yaxis.set_major_locator(ticker.MultipleLocator(1))

    plt.show()
    fig.savefig(input_sentence+'.png')


def evaluateAndShowAttention(input_sentence):
    output_words, attentions = evaluate(
        encoder1, attn_decoder1, input_sentence)
    print('input =', input_sentence)
    print('output =', ' '.join(output_words))
    showAttention(input_sentence, output_words, attentions)


evaluateAndShowAttention("my cats by my yaks dont smile quest")



input = my cats by my yaks dont smile quest
output = dont my cats by my yaks smile dont smile read smile smile sleep smile my cats smile dont smile read smile confuse my yaks that dont smile do irritate my cats that dont smile do smile read smile irritate my seals that dont smile do irritate my cats that dont smile do irritate my yaks that dont smile do confuse my cats that dont smile do irritate my seals that dont smile do confuse my yaks that dont smile do irritate my cats that dont smile do confuse my seals that dont smile do irritate my monkey that dont smile
